|               | actual cat | actual dog |
|:------------  |-----------:|-----------:|
| predicted cat |         34 |          7 |
| predicted dog |         13 |         46 |



In [2]:
true_positives = 34
true_negatives = 46
false_positives = 7
false_negatives = 13

# number of TRUE predictions (pos/neg) / total observations
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

# true positives / (all actual positives)
recall = true_positives / (true_positives + false_negatives)

# true positives / (all actual positive predictions)
precision = true_positives / (true_positives + false_positives)

# True Negative Rate
# true negatives / (all actual negatives) == TN / (TN + FP)
